In [2]:
# !pip install pandas
# !pip install d2lzh

%matplotlib inline
import d2lzh as d2l 
from mxnet import autograd, gluon, init, nd 
from mxnet.gluon import data as gdata, loss as gloss, nn 
import numpy as np
import pandas as pd

import pickle

# Load X_2020
with open('X_2020.pkl', 'rb') as file:
    X_2020 = pickle.load(file)
    print("success")

# Load y_2020
with open('y_2020.pkl', 'rb') as file:
    y_2020 = pickle.load(file)
    print("success")


/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


success
success


In [4]:

loss = gloss.L2Loss()

def get_net():
    net = nn.Sequential()
    
    # 第一层
    net.add(nn.Dense(14, activation='relu'))  
    # 第二层 
    net.add(nn.Dense(4, activation='relu'))  
    # 输出层
    net.add(nn.Dense(1))  # 输出层, 1个神经元
    net.initialize()
    return net


import mxnet.ndarray as nd

def binary_cross_entropy(preds, labels):
    # 将预测值进行限制，避免出现 log(0)
    preds = nd.clip(preds, 1e-10, 1 - 1e-10)
    return -nd.mean(labels * nd.log(preds) + (1 - labels) * nd.log(1 - preds))

def log_bce(net, features, labels):
   
    preds = net(features)  
    
    # 计算损失
    bce_loss = binary_cross_entropy(preds, labels)
    return bce_loss.asscalar()  

def calculate_accuracy(net, features, labels):
    # 将预测值二值化，使用0.5作为阈值
    preds = net(features)
    predicted_labels = (preds > 0.5).astype(labels.dtype)
    # 计算准确率
    return nd.mean(predicted_labels == labels).asscalar()


In [5]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls, accura_ls= [], [], []
    train_iter = gdata.DataLoader(gdata.ArrayDataset(
        train_features, train_labels), batch_size, shuffle=True)
    # print("train_iter",train_iter)
    # 这里使用了Adam优化算法
    trainer = gluon.Trainer(net.collect_params(), 'adam', {
        'learning_rate': learning_rate, 'wd': weight_decay})
    # print("trainer:",trainer)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        train_ls.append(log_bce(net, train_features, train_labels))
        accura_ls.append(calculate_accuracy(net, train_features, train_labels))
        # print("train_ls in loop:",train_ls)
        if test_labels is not None:
            test_ls.append(log_bce(net, test_features, test_labels))
        # print("train_ls1:",train_ls)
    return train_ls, test_ls,accura_ls


In [6]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = nd.concat(X_train, X_part, dim=0)
            y_train = nd.concat(y_train, y_part, dim=0)
    return X_train, y_train, X_valid, y_valid

In [7]:
def k_fold(k, X_train, y_train, num_epochs,
           learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum, accura_l_sum = 0, 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls,accura_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        accura_l_sum += accura_ls[-1]
        # print(valid_ls)
        # print("train_ls",train_ls)
        if i == 0:
            d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse',
                         range(1, num_epochs + 1), valid_ls,
                         ['train', 'valid'])
        print('fold %d, train loss%f, valid loss %f, accurate %f'
              % (i, train_ls[-1], valid_ls[-1], accura_ls[-1]))
    return train_l_sum / k, valid_l_sum / k, accura_l_sum / k


In [8]:

k, num_epochs, lr, weight_decay, batch_size = 5, 220, 0.00005, 0, 32
train_l, valid_l, accuracy_l = k_fold(k, X_2020, y_2020, num_epochs, lr,
                          weight_decay, batch_size)
print('%d-fold validation: avg train loss %f, avg valid loss %f, accuracy %f'
      % (k, train_l, valid_l, accuracy_l))
      

InvalidIndexError: (slice(0, 116968, None), slice(None, None, None))

# MLP finding
感觉准确性不是很高，也有可能是因为网络设计或者手动调参数的问题，这个太容易梯度消失了，lr大了容易梯度消失，小了容易loss变大，试一下Random forest + GridSearchCV


In [19]:
import pandas as pd
import numpy as np
from mxnet import nd, autograd, gluon

def get_k_fold_data(k, i, X, y):
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part = nd.array(X.iloc[idx].values)  # Convert DataFrame to NDArray
        y_part = nd.array(y.iloc[idx].values)  # Convert DataFrame to NDArray
        
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = nd.concat(X_train, X_part, dim=0)  # Concatenate NDArray
            y_train = nd.concat(y_train, y_part, dim=0)  # Concatenate NDArray
            
    return X_train, y_train, X_valid, y_valid

def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls, accura_ls = [], [], []
    train_iter = gluon.data.DataLoader(gluon.data.ArrayDataset(train_features, train_labels), 
                                       batch_size, shuffle=True)
    
    trainer = gluon.Trainer(net.collect_params(), 'adam', 
                            {'learning_rate': learning_rate, 'wd': weight_decay})
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = gloss.L2Loss()(net(X), y)
            l.backward()
            trainer.step(batch_size)
    
        train_ls.append(log_rmse(net, train_features, train_labels))
        accura_ls.append(calculate_accuracy(net, train_features, train_labels))
        
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels)) # type: ignore
    
    return train_ls, test_ls, accura_ls

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum, accura_l_sum = 0, 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls, accura_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        accura_l_sum += accura_ls[-1]
        
        if i == 0:
            d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse',
                         range(1, num_epochs + 1), valid_ls,
                         ['train', 'valid'])
        
        print('fold %d, train loss %f, valid loss %f, accuracy %f'
              % (i, train_ls[-1], valid_ls[-1], accura_ls[-1]))
    
    return train_l_sum / k, valid_l_sum / k, accura_l_sum / k

# 之后调用 k_fold 函数
k, num_epochs, lr, weight_decay, batch_size = 5, 220, 0.05, 0, 160
X_2020 = X_2020.astype(np.float32)
y_2020 = y_2020.astype(np.float32)
train_l, valid_l, accuracy_l = k_fold(k, X_2020, y_2020, num_epochs, lr, 
                                      weight_decay, batch_size)
print('%d-fold validation: avg train loss %f, avg valid loss %f, accuracy %f'
      % (k, train_l, valid_l, accuracy_l))

MXNetError: Traceback (most recent call last):
  File "../src/c_api/c_api_ndarray.cc", line 59
MXNetError: Check failed: inp->shape().Size() < (int64_t{1} << 31) - 1 (218904208384 vs. 2147483647) : [SetNDInputsOutputs] Size of tensor you are trying to allocate is larger than 2^31 elements. Please build with flag USE_INT64_TENSOR_SIZE=1